In [1]:
import numpy as np
import pandas as pd
import itertools

In [2]:
#CREATE THE DECK
list_of_numbers = range(2,15) # 2-10 + Jack=11, Queen=12, King=13, Ace=14
list_of_suits = ["S", "C", "D", "H"] #spades, clubs, diamonds, hearts
deck = []

for i in list_of_numbers:
    for l in list_of_suits:
        card = str(i)+l #because can't concatenate string + integers
        deck.append(card)
len(deck)

52

In [3]:
#CREATE ALL POSSIBLE COMBINATIONS OF HANDS.
##Formula of Combinations = n! / (r! (n -- r)!), where n is the total number of possibilities to start 
#and r is the number of selections made. This formula is calculated by itertools.combinations(n,r)
all_possible_combinations = []
for subset in itertools.combinations(deck, 5): 
    all_possible_combinations.append(subset)

In [4]:
#So, we have a deck. Now we need to set up the rules of poker, and the main thing are of course the
#COMBINATIONS(in order from weakest to strongest hand: highest card, pair, two pairs, three of a kind, straight, flush,
#full house, four of a kind, straight flush, royal flush)
# and
#SCORING


#COMBINATIONS 

#first things first: we have to write a function that recognizes the numbers and the suits of the cards.
# example of a hand that will be put as argument: hand = ['4C', '10C', '11C', '5C', '7C']
# this is a FLUSH, all 5 cards (jack, 10, 7, 5 and 4) are of the same suit "club".
# we want to have a function that splits each card's numbers from the letters
def get_nums_and_letters(hand):
    num = []
    lett = []
    i = 0
    while i<5:
        num.append(int(hand[i][:-1])) #from inner to outer parenthesis: we pick each cards number, 
        #transform it in intteger (because before we transformed it in string), and append to the list "num"
        lett.append(hand[i][-1]) #append each card's letter to the list "lett"
        i += 1
    return num, lett

#ok, let's take now another hand as example. Hand = ['4C', '4H', '11C', '5C', '4D'], which is of course a 
# "three of a kind" combination. Not a bad poker combination. 
#There is one last step to take before digging into coding a function that recognizes the combinations of
#any hand, and that is first writing a function that calculates the number of maximum recurrency of a 
#number in our hand (in our example is 3, because we have 3 fours), and the same regarding the letters (in
#our case it's 3, because we have 3 clubs).
#Why do we need this? Because for example if our program knows that the most frequent number is present 3 times
#like in our example, we can teach him that we therefore have either a "three of a kind", or a "full house".

def frequency_numbers_and_letters(num, lett):
    most_recurrent_nums = []
    for n in set(num): #set(num) = gives the unique values of the list "num". So if num = [4, 4, 4, 11, 10], 
                       # 4, 11, 10 are the unique values gathered by set(num)
        most_recurrent_nums.append(int(num.count(n))) #for each value in the list of unique values given by
        #set(num), count how many times this value is present in our hand. Add this number to the 
        #most_recurrent_nums list. So if num = [4, 4, 4, 11, 10], most_recurrent_nums = [3, 1, 1], because
        # our hand has 3 cards of 4, 1 jack, and 1 ten.
        
    most_frequent_number = max(most_recurrent_nums) #Take the highest number from "most_recurrent_nums".
    #In the example it is 3, because we have 3 cards that are of the same number (four).
    #It means that our hand either is a "three ok a kind", or a "full house".
    
    #we do the same thing for the letters
    most_recurrent_letts = []
    for l in set(lett):
        most_recurrent_letts.append(lett.count(l)) 
    most_frequent_letter = max(most_recurrent_letts) 
    
    return most_frequent_number, most_frequent_letter 
#just to avoid misunderstaind: 
#most_frequent_number = is not the card number of the most frequent number, but the FREQUENCY of the most frequent number
#most_frequent_letter = FREQUENCY of the most frequent letter
#if num = [12C, 12C, 4C, 11C, 10C]: most_frequent_number = 2, most_frequent_letter = 5


#GREAT! we can now code all of the possible combinations.
#highest card - pair - two pairs - three of a kind - straight - flush - full house - 4 of a kind - straight flush - royal flush

#So, the way we will code this function is dividing in two: the flush combinations (all cards of the same suit), 
#and the number combinations (where some numbers are recurrent and form a full house, or a three of a kind, etc).
#We can do this because IF we have any number combination, we CANT have a flush combination, simply because even a combination
#of just a pair (5C, 5D, 10C, 11C, 14C) can't ALSO be a flush, because there are two cards of the same number, and these
#two cards will for sure have different suits.

def best_combination(num, lett, most_frequent_number, most_frequent_letter):
   
    #so we will tell the algorithm to first check if most_frequent_letter = 5 (we coded it above, it means that all of 
    #the cards have the same letter/suit). If this condition is true, the only possible combinations we can have are: 
    #flush, straight flush, and royal flush.
    if most_frequent_letter == 5:
        if sorted(num) == [2, 3, 4, 5, 14]: 
            combination = "straight flush" #this is because in poker, ace is usually the largest card, except in a 
            #straight or straight flush combination, where it can be both 1 and 14. Indeed, A, 2, 3, 4, 5 forms a straight combination
        else:
            i = 1
            while i<5:
                if (sorted(num)[i] == sorted(num)[i-1]+1) == True:
                    if i == 4:
                        if sorted(num)[i] == 14:
                            combination = "royal flush"
                        else:
                            combination = "straight flush"
                        i += 1
                    else:
                        i += 1
                else:
                    combination = "flush" 
                    break
                

    #If the condition most_frequent_letter = 5 is False, we will tell the algorithm to check all the number combinations.
    #How? Well, using the most_frequent_number we coded in the previous function

    #IF most_frequent_number = 1, we have either a "highest card" or "straight" (remember, we can't have flush because the
    #function already checked this case)
    #IF most_frequent_number = 2, we have either "pair" or "two pairs"
    #IF most_frequent_number = 3, we have either a "three of a kind" or "full house"
    #IF most_frequent_number = 4, we have a "4 of a kind"
    else:
        if most_frequent_number == 1:
            i = 1
            while i < 5:
                if (sorted(num)[i] == sorted(num)[i-1]+1) == True or sorted(num) == [2, 3, 4, 5, 14]:
                    if i == 4:
                        combination = "straight"
                        i += 1
                    else:
                        i += 1
                else:
                    combination = "highest card"
                    break
 

        if most_frequent_number == 2:
            numbers_that_form_pairs = []
            for i in num:
                if num.count(i) == 2:
                    if i not in numbers_that_form_pairs:
                        numbers_that_form_pairs.append(i)
                    else:
                        pass
                else:
                    pass
            if len(numbers_that_form_pairs) == 1:
                combination = "pair"
            elif len(numbers_that_form_pairs) == 2:
                combination = "two pairs"
            
            
            
        if most_frequent_number == 3:
            numbers_that_form_pairs = []
            for i in num:
                if num.count(i) == 2:
                    if i not in numbers_that_form_pairs:
                        numbers_that_form_pairs.append(i)
                    else:
                        pass
                else:
                    pass
            if len(numbers_that_form_pairs) == 1:
                combination = "full house"
            elif len(numbers_that_form_pairs) == 0:
                combination = "three of a kind"
            
            
        if most_frequent_number == 4:
            combination = "four of a kind"
    return combination, num, lett

In [5]:
#SCORING

#in order to check whether a player's hand is winner over another hand, we need to compare them with a score.
#The scoring system I set up assigns from 1 point to 99 points. So each of the 2million+ possible hands, have a 
#scoring that goes from 1 to 99.

#The worst possible hand in poker is 2 3 4 5 7, of ≠ suits. The combination is "highest card", and the highest card
#in this case is 7. So, this hand takes the lowest point: 1. 
#The best possible "highest card" hand is the one where the highest card is Ace. Since Ace is 14 for us 
#(K=13, Q=12, J=11), a "highest card" of Ace will be valued 8 points. -> example ->
#-> 7 "highest card" = 1 point | 8 "highest card" = 2 points | 9 "highest card" = 3 points .... Ace "highest card" = 8 points
#This is exactly the way how I valued all of the combinatons, and below there is a quick summary:

#     -highest card
#     1 pto - 8 pti

#     -pair
#     9 pti - 21 pti
#     
#     -two pairs
#     22 pti - 33 pti
#     
#     -three of a kind
#     34 pti - 46 pti
#     
#     -straight (ACE CAN BE LOWEST CARD)
#     47 pti - 56 pti
#     
#     -flush
#     57 pti - 64 pti
#     
#     -full house
#     65pti - 77pti
#     
#     -four of a kind
#     78 pti - 90 pti
#     
#     -straight flush (ACE CAN BE LOWEST CARD)
#     91 pti - 98 pti
#     
#     -royal flush
#     99 pti

#Note: each combination's score will not actually be an integer, but a float. This because
#each score is determined by the value of the combination (i.e. a "three of a kind" like 2 2 2 Q 9 is 34 points) 
# + a float given by the value of the cards that don't form the combination.). The float value is necessary, otherwise
# a combination of 2 2 2 Q 9 and a combination of 2 2 2 5 3 would have the same score = 34, even though the first
# hand is of course the winner. Therefore, we need to evaluate also the other two cards that don't form the combination,
#and we translate them into floats by dividing them by 100 or 1000 or 10000 etc. So 2 2 2 Q 9 will have 
# a score = 34.1245 and 2 2 2 5 3 will have a score = 34.0515, declaring the first hand as winner.


def scoring(combination, num, lett):
    if combination == "highest card":
        score = sorted(num, reverse=True)[0] - 6 + sorted(num, reverse=True)[1]/100 + \
                sorted(num, reverse=True)[2]/1000 + sorted(num, reverse=True)[3]/10000 + sorted(num, reverse=True)[4]/100000
        
    if combination == "pair":
        pair = []
        no_pair = []
        for i in num:
            if num.count(i) == 2:
                if i not in pair:
                    pair.append(i)
                else:
                    pass
            else:
                no_pair.append(i)
        score = 9 + (pair[0] - 2) + sorted(no_pair, reverse=True)[0]/100 + sorted(no_pair, reverse=True)[1]/1000 + \
                sorted(no_pair, reverse=True)[2]/10000
        
    if combination == "two pairs":
        pairs = []
        no_pairs = []
        for i in num:
            if num.count(i) == 2:
                if i not in pairs:
                    pairs.append(i)
                else:
                    pass
            else:
                no_pairs.append(i)
        score = 22 + (max(pairs) - 3) + min(pairs)/50 + no_pairs[0]/2000
        
        
    if combination == "three of a kind":
        tris = []
        no_tris = []
        for i in num:
            if num.count(i) == 3:
                if i not in tris:
                    tris.append(i)
                else:
                    pass
            else:
                no_tris.append(i)
        score = 34 + (tris[0]-2) + sorted(no_tris, reverse=True)[0]/100 + \
                sorted(no_tris, reverse=True)[1]/2000
                
                
    if combination == "straight":
        if sorted(num) == [2, 3, 4, 5, 14]:
            score = 47
        else:
            score = 47 + (max(num) - 5)
        
        
    if combination == "flush":
        score = 57 + (sorted(num, reverse=True)[0]-7) + sorted(num, reverse=True)[1]/100 + \
                sorted(num, reverse=True)[2]/1500 + sorted(num, reverse=True)[3]/20000 + sorted(num, reverse=True)[4]/200000
        

    if combination == "full house":
        for i in num:
            if num.count(i) == 3:
                tris_number = i
            elif num.count(i) == 2:
                pair_number = i
        score = 65 + (tris_number - 2) + pair_number/100
        
        
    if combination == "four of a kind":
        for i in num:
            if num.count(i) == 4:
                four = i
            else:
                card = i
        score = 78 + (four-2) + card/100
                
                
    if combination == "straight flush":
        if sorted(num) == [2, 3, 4, 5, 14]:
            score = 91
        else:
            score = 92 + (max(num) - 7)
        
        
    if combination == "royal flush":
        score = 99
    return score

In [6]:
#Function that joins all of the above functions (combination + scoring)
def ultimate_function(hand):
    num, lett = get_nums_and_letters(hand)
    most_frequent_number, most_frequent_letter = frequency_numbers_and_letters(num, lett)
    combination, num, lett = best_combination(num, lett, most_frequent_number, most_frequent_letter)
    score = scoring(combination, num, lett)
    return combination, score

In [8]:
score_and_combination_column = []
for combination in all_possible_combinations:
    score_and_combination_column.append(ultimate_function(combination))

In [9]:
score_and_combination_column[:5]

[('four of a kind', 78.03),
 ('four of a kind', 78.03),
 ('four of a kind', 78.03),
 ('four of a kind', 78.03),
 ('four of a kind', 78.04)]

In [12]:
score_and_combination_df = pd.DataFrame(score_and_combination_column)
score_and_combination_df[:5]

,0,1
0,four of a kind,78.03
1,four of a kind,78.03
2,four of a kind,78.03
3,four of a kind,78.03
4,four of a kind,78.04


In [13]:
FINAL_COLUMN = pd.DataFrame({'Hand': all_possible_combinations,
                   'Combination': score_and_combination_df[0],
                   'Score': score_and_combination_df[1]})

In [14]:
FINAL_COLUMN[1500000:1500005]

,Hand,Combination,Score
1500000,"(3H, 8D, 11D, 13C, 14H)",highest card,8.14183
1500001,"(3H, 8D, 11D, 13D, 13H)",pair,20.11830
1500002,"(3H, 8D, 11D, 13D, 14S)",highest card,8.14183
1500003,"(3H, 8D, 11D, 13D, 14C)",highest card,8.14183
1500004,"(3H, 8D, 11D, 13D, 14D)",highest card,8.14183


In [23]:
#let's group by each poker combination and see how many hands out of all the 2'598'960 are of each combo.
combo_df = pd.DataFrame(FINAL_COLUMN["Combination"].groupby(FINAL_COLUMN["Combination"]).count())

In [16]:
combo_df

,Combination
Combination,
flush,5108
four of a kind,624
full house,3744
highest card,1302540
pair,1098240
royal flush,4
straight,10200
straight flush,36
three of a kind,54912


In [22]:
#just to check if it's all good
combo_df["Combination"].sum(), len(all_possible_combinations)

(2598960, 2598960)

In [18]:
percentage_combos = []
for element in combo_df["Combination"]:
    percentage_combos.append(element/len(all_possible_combinations)*100)

In [19]:
percentage_combos = list(percentage_combos)
percentage_combos

[0.1965401545233478,
 0.024009603841536616,
 0.14405762304921968,
 50.11773940345369,
 42.25690276110444,
 0.000153907716932927,
 0.39246467817896385,
 0.0013851694523963432,
 2.112845138055222,
 4.75390156062425]

In [20]:
combo_df["Percentage combos"] = percentage_combos

In [21]:
combo_df

,Combination,Percentage combos
Combination,,
flush,5108,0.196540
four of a kind,624,0.024010
full house,3744,0.144058
highest card,1302540,50.117739
pair,1098240,42.256903
royal flush,4,0.000154
straight,10200,0.392465
straight flush,36,0.001385
three of a kind,54912,2.112845
